# Snowflake real-world project case study #

## Goal ##
- Centralize data
- Enable Real-time analytics
- Improve scalability
- Enhance Data Quality
- Support advanced analytics


## Overall Architecture:
- Data sources (CSV, JSON, Parquet)
- Data storage (ADLS)
- Data warehouse (Snowflake)
- Data processing layers (Bronze, Silver, Gold)
- Reporting and Analytics tools

## Data Flow ##
- Data  source systems → ADLS (External Stages)
- ADLS → Bronze (COPY command)
- Bronze → Silver (Transformations and quality rules)
- Silver → Gold layer (Business specific aggregates)
- Gold → BI Tools (Reporting and Analytics)


## Key Snowflake Features ##
- External stages → (to connect ADLS account to the snowflake to pull the data)
- COPY command → (to load the data)
- Tasks → (to automate data loading and processing workflows)
- Streams → (to capture and process incremental changes)
- Time Travel → (for data recovery and historical analysis)
- Zero copy cloning → (for efficient environment provisioning for dev, test and prod)


## Key Components ##
- Snowflake DB
- External stage
- Multiple schema (bronze, silver, gold)
- Tables
- Tasks
- Streams
- Views


## Implementation ##

### Create External Stage ###

In [ ]:
# Create storage integration to ADLS
# A storage integration is a secure object that connects Snowflake to external cloud storage
CREATE OR REPLACE STORAGE INTEGRATION <integration_name>
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = AZURE
  ENABLED = TRUE
  AZURE_TENANT_ID = 'tenant_ID'
  STORAGE_ALLOWED_LOCATIONS = ('azure://<azure_storage_acct>.blob.core.windows.net/<container>/');

# Show me the configuration details of the storage integration (It returns metadata about the integration)
DESC STORAGE INTEGRATION <integration_name>;
    /*
    AZURE_CONSENT_URL:
        - is a Microsoft Entra (Azure AD) consent link that an Azure administrator must open to grant Snowflake permission to access Azure storage
        - Opens an Azure permission screen
        - Lets an admin approve Snowflake as an application
        - Allows access to Azure storage without passwords or shared keys

    AZURE_MULTI_TENANT_APP_NAME:
        - It is the name of the Azure (Entra ID) application registered by Snowflake that enables secure access to your Azure storage account.
        - It identifies the application identity Snowflake uses inside Microsoft Entra ID (Azure AD) to authenticate.
    */

### Grant Read/Write permission on ADSL to snoeflake application (AZURE_MULTI_TENANT_APP_NAME) ##
- Storage account → Access control (IAM) → Add → Role assignment → Storege blob data contributer → Select application → Review + Assign

In [ ]:
# Create snowflake DB
CREATE DATABASE IF NOT EXISTS <database_name>;

# Create bronze schema:
CREATE SCHEMA IF NOT EXISTS bronze;

# Create stage (in order to create the stage we need to first create the database in snowflake)
USE p<database_nam>.<schema>
CREATE OR REPLACE STAGE <stage_name>
    STORAGE_INTEGRATION = <integration_name>
    URL = 'azure://<azure_storage_acct>ilstg.blob.core.windows.net/<container>';

-- check the stage:
-- LS @stage is a command used to list files stored in a stage (a staging location for data files before loading or unloading data).
ls @<stage_name>